In [1]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [2]:
# import mindpype
import mindpype as mp
import numpy as np

In [3]:
# get the training files
from glob import glob
training_files = glob("P:/general_prism/Side Projects/Summer Student Onboarding project 2024/Data/JerrySelf/*.xdf")
print(training_files)


['P:/general_prism/Side Projects/Summer Student Onboarding project 2024/Data/JerrySelf\\sub-P004_ses-S001_task-vP300+2x2_run-007.xdf', 'P:/general_prism/Side Projects/Summer Student Onboarding project 2024/Data/JerrySelf\\sub-P004_ses-S001_task-vP300+2x2_run-008.xdf', 'P:/general_prism/Side Projects/Summer Student Onboarding project 2024/Data/JerrySelf\\sub-P004_ses-S001_task-vP300+2x2_run-009.xdf']


In [4]:
# create the mindpype session
s = mp.Session.create()

In [5]:
# define some session parameters
Fs = 250
Nc = 32
trial_len = 1
Ns = int(trial_len * Fs)

start_time = 0.0
end_time = 1.0

start_ix = int(start_time * Fs) + len(f.coeffs["fir"])
end_ix = int(np.ceil(end_time * Fs)) + len(f.coeffs["fir"])
extract_indices = [
    slice(None),
    slice(start_ix, end_ix),
]  # All epochs, all channels, start_time to end_time

In [6]:
# create the data sources

# training data
tr_data_src = mp.source.InputXDFFile.create_epoched(s, training_files, channels=range(Nc),
                                                    stype='EEG', Ns=Ns,
                                                    relative_start=-0.2)

t_tr_data, t_tr_labels = tr_data_src.load_into_tensors()

c:\Users\ivanovn\Documents\MindPype\mindpype\source.py:158: RuntimeWarning: No task list provided. Infering tasks from the marker stream. This is only supported for Mindset P300 data.
  marker_stream = self._filter_marker_stream(marker_stream)


In [7]:
# create a filter
bandpass = (1, 25)  # in Hz
f = mp.Filter.create_fir(
    s, Fs, bandpass[0], bandpass[1], method="fir", phase="minimum"
)

# create the classifier
classifier = mp.Classifier.create_logistic_regression(s)

# create the graph
g = mp.Graph.create(s)

No data specified. Sanity checks related to the length of the signal relative to the filter order will be skipped.
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 1.00 Hz
- Upper transition bandwidth: 6.25 Hz
- Filter length: 825 samples (3.300 s)



In [8]:
# test data
te_data_src = mp.source.InputLSLStream.create_marker_coupled_data_stream(s, pred="type='EEG'", channels=range(Nc),
                                                                         relative_start=-0.2, Ns=Ns,
                                                                         marker_fmt='.*flash',
                                                                         marker_pred="type='Markers'")

t_input_data = mp.Tensor.create_from_handle(s, (Nc, Ns), te_data_src)

c:\Users\ivanovn\Documents\MindPype\mindpype\source.py:585: RuntimeWarning: More than one stream found matching the predicate. Using the first stream found.
  self.update_input_streams(pred, channels, marker_coupled, marker_fmt, marker_pred, stream_info, marker_stream_info, Ns)
c:\Users\ivanovn\Documents\MindPype\mindpype\source.py:585: RuntimeWarning: More than one marker stream found matching the predicate. Using the first stream found.
  self.update_input_streams(pred, channels, marker_coupled, marker_fmt, marker_pred, stream_info, marker_stream_info, Ns)


In [9]:

t_pred_label = mp.Tensor.create(s, (1,))
t_pred_probs = mp.Tensor.create(s, (1, 2))

In [10]:
v_tensors = [
                mp.Tensor.create_virtual(s),  # 0 - output of pad, input to filter
                mp.Tensor.create_virtual(s),  # 1 - output of filter, input to extract
                mp.Tensor.create_virtual(s),  # 2 - output of extract, input to baseline_correction
                mp.Tensor.create_virtual(s),  # 3 - output of baseline correction, input to resample
                mp.Tensor.create_virtual(s),  # 4 - output of xdawn, input to tangent space
                mp.Tensor.create_virtual(s),  # 5 - output of tangent space, input to classifier
            ]

In [11]:
# Add nodes to the graph

mp.kernels.PadKernel.add_to_graph(
    g,
    t_input_data,
    v_tensors[0],
    pad_width=((0, 0), (len(f.coeffs["fir"]), len(f.coeffs["fir"]))),
    mode="edge",
    init_input=t_tr_data, init_labels=t_tr_labels
)

mp.kernels.FilterKernel.add_to_graph(
    g, 
    v_tensors[0], 
    f, 
    v_tensors[1], 
    axis=1,
)

mp.kernels.ExtractKernel.add_to_graph(
    g, 
    v_tensors[1], 
    extract_indices, 
    v_tensors[2]
)

mp.kernels.BaselineCorrectionKernel.add_to_graph(
    g,
    v_tensors[2],
    v_tensors[3],
    baseline_period=[0 * Fs, 0.2 * Fs],
)

mp.kernels.XDawnCovarianceKernel.add_to_graph(
    g,
    v_tensors[3],
    v_tensors[4],
    num_filters=4,
    estimator="lwf",
    xdawn_estimator="lwf",
)
mp.kernels.TangentSpaceKernel.add_to_graph(
    g, 
    v_tensors[4], 
    v_tensors[5], 
    metric="riemann"
)

mp.kernels.ClassifierKernel.add_to_graph(
    g, v_tensors[5], 
    classifier, 
    t_pred_label, 
    t_pred_probs
)

In [12]:
# verify
g.verify()

# cross validate and init graph
cv_acc = g.cross_validate(t_pred_label)
print(f"Cross validation accuracy {cv_acc:0.3f}")

g.initialize()

Verifying kernel Pad...
Verifying kernel Filter...
Verifying kernel Extract...
Verifying kernel BaselineCorrection...
Verifying kernel XDawnCovarianceKernel...
Verifying kernel TangentSpaceKernel...
Verifying kernel Classifier...
Cross validation accuracy 0.950


In [13]:
# execute the graph
while True:
    g.execute()
    print(f"Predicted label: {t_pred_label.data[0]}, predicted probabilities: {t_pred_probs.data[0]}")

Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.73762444 0.26237556]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.76522348 0.23477652]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.82266118 0.17733882]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.81660115 0.18339885]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.75850197 0.24149803]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.71669301 0.28330699]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.88492984 0.11507016]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.80309036 0.19690964]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.72621807 0.27378193]


c:\Users\ivanovn\Documents\MindPype\mindpype\containers.py:752: RuntimeWarning: Buffer contains a backlog of data. Execution may be too slow for target polling rate.
  data = self.ext_src.poll_data(label=label)
c:\Users\ivanovn\Documents\MindPype\mindpype\containers.py:752: RuntimeWarning: Buffer contains a backlog of data. Execution may be too slow for target polling rate.
  data = self.ext_src.poll_data(label=label)
c:\Users\ivanovn\Documents\MindPype\mindpype\containers.py:752: RuntimeWarning: Buffer contains a backlog of data. Execution may be too slow for target polling rate.
  data = self.ext_src.poll_data(label=label)
c:\Users\ivanovn\Documents\MindPype\mindpype\containers.py:752: RuntimeWarning: Buffer contains a backlog of data. Execution may be too slow for target polling rate.
  data = self.ext_src.poll_data(label=label)
c:\Users\ivanovn\Documents\MindPype\mindpype\containers.py:752: RuntimeWarning: Buffer contains a backlog of data. Execution may be too slow for target poll

Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.74274166 0.25725834]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.70791799 0.29208201]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.77641541 0.22358459]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.81380591 0.18619409]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.80023476 0.19976524]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.89503096 0.10496904]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.86134075 0.13865925]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.71829163 0.28170837]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.68106543 0.31893457]
Executing trial with label: None
Predicted label: 0, predicted probabilities: [0.6

KeyboardInterrupt: 